In [1]:
import os

# spark_version
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-17 22:05:15--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.56MB/s    in 0.2s    

2021-11-17 22:05:16 (4.56 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# EXTRACT


In [4]:
# Read in data from S3 Bucket
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
kitchen_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), sep="\t",header=True, inferSchema=True)

# Show DataFrame
kitchen_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [5]:
# Count the number of records
kitchen_df.count()

4880466

# TRANSFORM

In [6]:
# Select "review_id", "customer_id", "product_id", "product_parent" and "review_date" columns
reviews_df = kitchen_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
reviews_df.show(truncate=False)

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3DT59XH7HXR9K|37000337   |B00303FI0G|529320574     |2015-08-31 |
|R1LFS11BNASSU8|15272914   |B00JCZKZN6|274237558     |2015-08-31 |
|R296RT05AG0AF6|36137863   |B00JLIKA5C|544675303     |2015-08-31 |
|R3V37XDZ7ZCI3L|43311049   |B000GBNB8G|491599489     |2015-08-31 |
|R14GU232NQFYX2|13763148   |B00VJ5KX9S|353790155     |2015-08-31 |
|RZQH4V7L2O1PL |19009420   |B00HYB5YY0|432241873     |2015-08-31 |
|R1F8JMOSPJ3KO7|40599388   |B000HEBAV2|584680984     |2015-08-31 |
|R1ZISGY2BWW4Z5|22719359   |B0012DS4GG|772637306     |2015-08-31 |
|R17PW4I3AE5WZW|47478640   |B00FLQ4EE6|755416578     |2015-08-31 |
|R3D93G1KTP6A8P|34195504   |B00DBS9OTG|648762742     |2015-08-31 |
|R18TQIW1NKPUNU|19100570   |B00AN9UJ68|495720940     |2015-08-31 |
|R34KUNL21WU248|10299811   |B00L2P0KNO|41330497      |2015-08-

In [7]:
# Drop duplicates and null values
reviews_df = reviews_df.drop_duplicates(["review_id"])
reviews_df = reviews_df.dropna()

In [8]:
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType

# Convert "review_date" column from string to date format
date_format = udf(lambda x: datetime.strptime(x, "%Y-%m-%d"), DateType())
reviews_df = reviews_df.withColumn("review_date", date_format(col("review_date")))

In [9]:
# 'review_id_table' 
reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
# Select "product_id" and "product_title" columns
products_df = kitchen_df.select(["product_id","product_title"])

# Drop duplicates and null values
products_df = products_df.drop_duplicates(["product_id"])
products_df = products_df.dropna()

# Show DataFrame
products_df.show(truncate=False)

+----------+------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                     |
+----------+------------------------------------------------------------------------------------------------------------------+
|0130350591|Le Creuset Kiwi (Green) Butter Dish Stoneware                                                                     |
|0594006112|Elf on the Shelf 10oz (Christmas Mug)                                                                             |
|0594255139|Shh I'm Reading Porcelain Coffee/TEA Mug                                                                          |
|0594456207|Hooty Owl Fold Over Lunch Tote 9" x 7" x 4.75"                                                                    |
|0641929307|Vera Bradley Blue Rhapsody Mug W/Lid                                                        

In [11]:
# 'products' table 
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [12]:
# unique customers and count of reviews
customers_df = kitchen_df.groupBy("customer_id").count()
customers_df = customers_df.withColumnRenamed("count","customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   23042837|             1|
|   48875707|             2|
|   48088660|             1|
|   20781887|             6|
|    1075003|             1|
|   28377689|             3|
|    8338749|             1|
|   39271457|             6|
|   38209321|             5|
|   51397605|             1|
|   45337932|             7|
|   12948675|             3|
|   38892468|             1|
|   43582450|             1|
|   13156755|             1|
|   13433330|             1|
|   17067926|             7|
|   13780617|             1|
|   50607818|             1|
|   33437759|             2|
+-----------+--------------+
only showing top 20 rows



In [13]:
# 'customers' table
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [14]:
# Select "review_id", "star_rating", "helpful_votes", "total_votes", "vine" columns
vine_df = kitchen_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])

# Make sure there are only unique id numbers and drop null values
vine_df = vine_df.drop_duplicates(["review_id"])
vine_df = vine_df.dropna()

vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10002O5HSLMDW|          5|            0|          0|   N|
|R100042H9DN6G5|          5|            0|          0|   N|
|R100065LWA66A9|          4|            0|          1|   N|
|R10006YK5ARTDT|          5|            0|          0|   N|
|R100079NQX5FV0|          5|            0|          0|   N|
|R10009BPC6YR2T|          4|            0|          0|   N|
|R1000CEJZ30MZL|          5|            1|          2|   N|
|R1000CK3G7C8XX|          5|            0|          0|   N|
|R1000IX1RKMQ8T|          5|            0|          0|   N|
|R1000JEOH5IPCA|          4|            0|          0|   N|
|R1000L0JTNVOMK|          5|            0|          0|   N|
|R1000MJ0FG0INR|          4|            0|          0|   N|
|R1000OO0PP3MXV|          5|            0|          0|   N|
|R1000OU7YF4TL6|          5|            

In [15]:
# 'vine_table' 
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



# LOAD


In [17]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [19]:
import sys
sys.path.append('/content/gdrive/MyDrive/Amazon Vine Program Analysis')

In [20]:
from config import endpoint, user_name, password
# Configure settings for RDS
mode1 = "append"
mode2 = "overwrite"
jdbc_url="jdbc:postgresql://" + endpoint + ":5432/VineProgramDB"
config = {"user": user_name, 
          "password": password, 
          "driver":"org.postgresql.Driver"}

In [21]:
# Write reviews_df to 'review_id_table' table in RDS
reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode1, properties=config)

In [22]:
# Write products_df to 'products' table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode1, properties=config)

In [23]:
# Write customers_df to 'customers' table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode1, properties=config)

In [24]:
# Write vine_df to 'vine_table' table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode1, properties=config)